# Pipeline de Pré-processamento

Este notebook descreve a etapa 2 do trabalho: limpeza, engenharia e transformação dos dados da Olist para gerar um conjunto adequado ao treinamento, sempre usando o dataset completo.

## Visão geral

1. Carregar as tabelas brutas da Olist e derivar a variável alvo e agregações.
2. Normalizar textos (maiúsculas, remoção de acentos, `_`).
3. Usar `ColumnTransformer` para imputação, escalonamento e codificação.
4. Exportar o resultado compacto em `data/clean/olist_ml_ready.csv`.
5. Visualizar indicadores com Plotly para toda a base.

### Bibliotecas e configurações

Este bloco importa geopandas, pandas, plotly e scikit-learn, além de definir os diretórios para leitura e escrita dos dados.

In [ ]:
import unicodedata
from pathlib import Path

import geopandas as gpd
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler

RAW_DIR = Path('/home/arthur/projects/data-eda-proj/data/raw')
CLEAN_DIR = Path('/home/arthur/projects/data-eda-proj/data/clean')
CLEAN_DIR.mkdir(exist_ok=True)
TARGET_FILE = CLEAN_DIR / 'olist_ml_ready.csv'


In [ ]:
def normalize_text(value):
    if pd.isna(value):
        return value
    text = str(value).strip().upper()
    stripped = ''.join(
        ch for ch in unicodedata.normalize('NFD', text)
        if unicodedata.category(ch) != 'Mn'
    )
    return stripped.replace(' ', '_')

def normalize_frame(dataframe):
    return dataframe.applymap(normalize_text)


### Carregando as tabelas brutas

Lê todas as CSVs oficiais da Olist que serão consolidadas no dataset final (pedidos, itens, clientes, sellers, geolocalizações, pagamentos e reviews).

In [ ]:
orders = pd.read_csv(
    RAW_DIR / 'olist_orders_dataset.csv',
    parse_dates=[
        'order_purchase_timestamp',
        'order_approved_at',
        'order_delivered_carrier_date',
        'order_delivered_customer_date',
        'order_estimated_delivery_date'
    ]
)
order_items = pd.read_csv(
    RAW_DIR / 'olist_order_items_dataset.csv',
    parse_dates=['shipping_limit_date']
)
customers = pd.read_csv(RAW_DIR / 'olist_customers_dataset.csv')
sellers = pd.read_csv(RAW_DIR / 'olist_sellers_dataset.csv')
geolocation = pd.read_csv(RAW_DIR / 'olist_geolocation_dataset.csv')
payments = pd.read_csv(RAW_DIR / 'olist_order_payments_dataset.csv')
reviews = pd.read_csv(RAW_DIR / 'olist_order_reviews_dataset.csv')
products = pd.read_csv(RAW_DIR / 'olist_products_dataset.csv')
category_translation = pd.read_csv(RAW_DIR / 'product_category_name_translation.csv')


### Engenharia temporal e agregações

Cálculo da variável alvo (`delivery_time_days`), atrasos de aprovação/estimativa e agregações por pedido (quantidade de itens, preços, categoria principal, pagamentos e reviews).

In [ ]:
orders = orders[orders['order_delivered_customer_date'].notna()].copy()
orders['delivery_time_days'] = (
    (orders['order_delivered_customer_date'] - orders['order_purchase_timestamp']).dt.total_seconds() / 86400
)
orders['approval_delay_hours'] = (
    (orders['order_approved_at'] - orders['order_purchase_timestamp']).dt.total_seconds() / 3600
)
orders['delivery_estimate_gap_days'] = (
    (orders['order_estimated_delivery_date'] - orders['order_delivered_customer_date']).dt.total_seconds() / 86400
)
orders = orders[orders['delivery_time_days'] >= 0]

category_translation = category_translation.fillna('UNKNOWN')
products = products.merge(
    category_translation, on='product_category_name', how='left'
)
products['product_category_name_english'] = products['product_category_name_english'].fillna('UNKNOWN').astype(str)
order_items = order_items.merge(
    products[['product_id', 'product_category_name_english']],
    on='product_id', how='left'
)
items_agg = (
    order_items.groupby('order_id', as_index=False)
    .agg(
        items_count=('order_item_id', 'count'),
        unique_sellers=('seller_id', 'nunique'),
        unique_products=('product_id', 'nunique'),
        total_price=('price', 'sum'),
        total_freight=('freight_value', 'sum')
    )
)
items_agg['price_per_item'] = items_agg['total_price'] / items_agg['items_count']
category_mode = (
    order_items.groupby('order_id')
    .agg(primary_category=('product_category_name_english', lambda s: s.mode().iloc[0] if not s.mode().empty else 'UNKNOWN'))
    .reset_index()
)
payment_agg = (
    payments.groupby('order_id', as_index=False)
    .agg(
        payment_value_sum=('payment_value', 'sum'),
        payment_installments_max=('payment_installments', 'max'),
        payment_type=('payment_type', 'first')
    )
)
review_agg = (
    reviews.groupby('order_id', as_index=False)
    .agg(
        average_review_score=('review_score', 'mean'),
        review_count=('review_id', 'count')
    )
)


### Combinação com geolocalizações

Integra as latitudes/longitudes do cliente e do seller usando a tabela de CEPs para habilitar análises espaciais e cálculo de distâncias.

In [ ]:
customers = customers.merge(
    geolocation.groupby('geolocation_zip_code_prefix', as_index=False)
    .agg(latitude=('geolocation_lat', 'median'), longitude=('geolocation_lng', 'median')),
    left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left'
)
customers.rename(columns={'latitude': 'customer_latitude', 'longitude': 'customer_longitude'}, inplace=True)

sellers = sellers.merge(
    geolocation.groupby('geolocation_zip_code_prefix', as_index=False)
    .agg(latitude=('geolocation_lat', 'median'), longitude=('geolocation_lng', 'median')),
    left_on='seller_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left'
)
sellers.rename(columns={'latitude': 'seller_latitude', 'longitude': 'seller_longitude'}, inplace=True)

seller_for_order = (
    order_items.sort_values('order_item_id')
    .drop_duplicates('order_id')
    .merge(
        sellers[['seller_id', 'seller_state', 'seller_latitude', 'seller_longitude']],
        on='seller_id', how='left'
    )
    .rename(columns={'seller_state': 'primary_seller_state'})
    [['order_id', 'primary_seller_state', 'seller_latitude', 'seller_longitude']]
)

dataset = (
    orders
    .merge(items_agg, on='order_id', how='left')
    .merge(category_mode, on='order_id', how='left')
    .merge(payment_agg, on='order_id', how='left')
    .merge(review_agg, on='order_id', how='left')
    .merge(
        customers[['customer_id', 'customer_state', 'customer_latitude', 'customer_longitude']],
        on='customer_id', how='left'
    )
    .merge(seller_for_order, on='order_id', how='left')
)
dataset['order_month'] = dataset['order_purchase_timestamp'].dt.month
dataset['order_weekday'] = dataset['order_purchase_timestamp'].dt.dayofweek


### Distâncias geográficas

Remove registros sem coordenadas completas e usa GeoPandas para calcular a distância real em quilômetros entre o cliente e o seller.

In [ ]:
dataset = dataset.dropna(subset=[
    'customer_latitude', 'customer_longitude', 'seller_latitude', 'seller_longitude'
]).copy()
print(f'Dataset limpo com {len(dataset)} linhas e {dataset.shape[1]} colunas.')

customer_points = gpd.GeoSeries(
    gpd.points_from_xy(dataset['customer_longitude'], dataset['customer_latitude']),
    crs='EPSG:4326'
)
seller_points = gpd.GeoSeries(
    gpd.points_from_xy(dataset['seller_longitude'], dataset['seller_latitude']),
    crs='EPSG:4326'
)
dataset['distance_km'] = (
    customer_points.to_crs('EPSG:3857')
    .distance(seller_points.to_crs('EPSG:3857'))
    / 1000
)

Dataset limpo com 95998 linhas e 31 colunas.


### Normalização textual

Aplica funções que convertem textos para maiúsculas, removem acentos e trocam espaços por `_` nos atributos categóricos antes da codificação.

In [ ]:
dataset.drop(columns=['customer_city', 'primary_seller_city'], inplace=True, errors='ignore')
categorical_columns = ['customer_state', 'primary_seller_state', 'payment_type', 'primary_category']
numeric_columns = [
    'items_count', 'unique_sellers', 'unique_products', 'total_price', 'total_freight',
    'price_per_item', 'payment_value_sum', 'payment_installments_max', 'average_review_score',
    'review_count', 'customer_latitude', 'customer_longitude', 'seller_latitude', 'seller_longitude',
    'order_month', 'order_weekday', 'distance_km'
]


### Pipeline de transformação

Configura o `ColumnTransformer` com pipelines separados para atributos categóricos (normalização, imputação e OneHot) e numéricos (imputação e `StandardScaler`).

In [ ]:
cat_pipeline = Pipeline([
    ('normalizer', FunctionTransformer(normalize_frame, validate=False)),
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
preprocessor = ColumnTransformer([
    ('categorical', cat_pipeline, categorical_columns),
    ('numerical', num_pipeline, numeric_columns)
])


### Aplicação final e exportação

Transforma as features, adiciona a variável alvo e grava o CSV pronto em `data/clean/olist_ml_ready.csv`.

In [ ]:
target = 'delivery_time_days'
feature_df = dataset[categorical_columns + numeric_columns].copy()
processed = preprocessor.fit_transform(feature_df)
cat_encoder = preprocessor.named_transformers_['categorical'].named_steps['encoder']
cat_columns_out = list(cat_encoder.get_feature_names_out(categorical_columns))
final_columns = cat_columns_out + numeric_columns
final_df = pd.DataFrame(processed, columns=final_columns, index=feature_df.index)
final_df[target] = dataset[target].values
final_df.to_csv(TARGET_FILE, index=False)
print('Pipeline concluída e exportada para', TARGET_FILE)


/tmp/ipykernel_61759/3644485920.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return dataframe.applymap(normalize_text)


AttributeError: Estimator normalizer does not provide get_feature_names_out. Did you mean to call pipeline[:-1].get_feature_names_out()?

### Distribuição da distância

Histograma interativo com Plotly mostrando como as distâncias client-seller se comportam na base inteira.

In [ ]:
fig = px.histogram(
    dataset,
    x='distance_km',
    nbins=60,
    title='Distribuição das distâncias (km) entre cliente e seller',
    color_discrete_sequence=['#636EFA']
)
fig.update_layout(bargap=0.1)
fig.show()


### Relação distância x tempo de entrega

Dispersão colorida por categoria principal para inspeção visual de possíveis padrões de atraso geográfico e por produto.

In [ ]:
fig2 = px.scatter(
    dataset,
    x='distance_km',
    y='delivery_time_days',
    color='primary_category',
    title='Tempo de entrega x distância (dados completos)',
    opacity=0.6,
    hover_data=['items_count', 'payment_type']
)
fig2.update_layout(height=600)
fig2.show()